In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import glob
import os
import os.path as osp
import shutil
import random
import math
import json
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

In [2]:
class_index = ['コンクリート', 'レール', '金枠あり', '背景あり']
weight_index = [0, 0, 0, 0]
batch_size = 16

データの前処理

In [3]:
class makeDataList():
    def __init__(self):
        self.rootpath =  './ImageData/'
        # 10枚に１枚が検証用データ
        self.trainPer = 10
    
    def shuffle_train_test(self):
        for path in glob.glob(self.rootpath+'/*'):
            print(path)

            trainpath = './train/'+path.split('/')[2]
            if not os.path.exists(trainpath):
                os.makedirs(trainpath)

            valpath = './val/'+path.split('/')[2]
            if not os.path.exists(valpath):
                os.makedirs(valpath)

            for j,image in enumerate(glob.glob(self.rootpath+path.split('/')[2]+'/*.jpg')):
                print(j)
                if j%self.trainPer==0:
                    shutil.copyfile(image, './val/'+path.split('/')[2]+'/'+image.split('/')[3])
                else:
                    shutil.copyfile(image, './train/'+path.split('/')[2]+'/'+image.split('/')[3])
                    
    def make_data_path(self, phase='train'):
        target_path = osp.join('./'+phase+'/**/*.jpg')
        path_list = []
        for path in glob.glob(target_path):
            path_list.append(path)
        
        return path_list

In [4]:
class BaseTransform():
    def __init__(self):
        self.size = 256
        self.mean = ((0.5,))
        self.std = ((0.22,))
        self.base_transform = {
            'train':
                transforms.Compose([
                transforms.RandomResizedCrop(self.size, scale=(0.5,1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)
                ]),
            'val':
                transforms.Compose([
                transforms.Resize(self.size),
                transforms.CenterCrop(self.size),
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)
                ])
        }
        
    def __call__(self, img, phase='train'):
        return self.base_transform[phase](img)

In [5]:
class HymenopteraDataset(data.Dataset):
    
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.phase)
        if self.phase == 'train':
            label = img_path.split('/')[2]
            label = class_index.index(label)
            weight_index[label] += batch_size
        elif self.phase == 'val':
            label = img_path.split('/')[2]
            label = class_index.index(label)
        
        return img_transformed, label

In [6]:
dataList = makeDataList()
#dataList.shuffle_train_test()
trainList = dataList.make_data_path(phase='train')
valList = dataList.make_data_path(phase='val')

In [7]:
trainDataset = HymenopteraDataset(file_list=trainList, transform=BaseTransform(), phase='train')
valDataset = HymenopteraDataset(file_list=valList, transform=BaseTransform(), phase='val')

In [8]:
index = 10
print(trainDataset.__getitem__(index)[0].size())
print(trainDataset.__getitem__(index)[1])

torch.Size([1, 256, 256])
0


In [9]:
print(weight_index)

[32, 0, 0, 0]


In [10]:
trainDataloader = torch.utils.data.DataLoader(trainDataset,
                                              batch_size=batch_size,
                                              shuffle=True)
valDataloader = torch.utils.data.DataLoader(valDataset,
                                            batch_size=batch_size,
                                            shuffle=False)
dataloaders_dict = { 'train': trainDataloader, 'val': valDataloader }
batch_iterator = iter(dataloaders_dict['train'])
inputs, labels = next(batch_iterator)

print(inputs.size())
print(labels)

torch.Size([16, 1, 256, 256])
tensor([1, 0, 0, 1, 1, 0, 0, 3, 3, 0, 2, 3, 0, 1, 0, 1])


In [11]:
new_weight_index = list(map(lambda x: float(math.ceil(sum(weight_index)/x)), weight_index))
new_weight_index

[2.0, 4.0, 18.0, 6.0]

GoogLeNetを使った学習

In [48]:
class ResNet():
    
    def __init__(self):
        self.net = models.resnet18(pretrained=True)
        self.net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.net.fc = nn.Linear(self.net.fc.in_features, len(class_index))
        self.net.train()
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.net.to(self.device)
        self.num_epochs = 30
        print('use in device: ', self.device)
        print(self.net.fc.out_features)
        weights = torch.tensor(new_weight_index).cuda()
        print(weights)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(params=self.net.parameters(), lr=0.001, momentum=0.9)
        self.lr_scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=7, gamma=0.1)
        
    def train(self, dataloaderDict):
        self.net.to(self.device)
        torch.backends.cudnn.benchmark = True
        for epoch in range(self.num_epochs):
            print('Epochs {}/{}'.format(epoch+1, self.num_epochs))
            for phase in ['train', 'val']:
                if phase == 'train':
                    self.lr_scheduler.step()
                    self.net.train()
                else:
                    self.net.eval()
                    
                epoch_loss = 0.0
                epoch_corrects = 0
                
                if (epoch == 0) and (phase == 'train'):
                    continue
                for inputs, labels in tqdm(dataloaderDict[phase]):
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    outputs = self.net(inputs)
                    loss = self.criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        self.optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    
                epoch_loss = epoch_loss / len(dataloaderDict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloaderDict[phase].dataset)
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                
    def eval(self, imgList, transform):
        
        
        for label in class_index:
            path = './result/'+label
            if not os.path.exists(path):
                os.makedirs(path)
        
        load_path = './weightsFineTuning.pth'
        load_weights = torch.load(load_path)
        
        self.net.load_state_dict(load_weights)
        self.net.load_state_dict(torch.load(load_path, map_location={'cuda:0': 'cpu'}))
        self.net.eval()
        
        for _img in imgList:
            img = Image.open(_img)
            img_transformed = transform(img)
            inputs = img_transformed.unsqueeze_(0)
            inputs = inputs.to(self.device)
            out = self.net(inputs)
            img.close()
            
            maxid = np.argmax(out.cpu().detach().numpy())
            predict_label_name = class_index[maxid]
            print(predict_label_name)
            filename = _img.split('/')[-1]
            shutil.copyfile(_img, './result/'+ predict_label_name+'/'+filename)
            
                
    def saveWeight(self):
        save_path = './weightsFineTuning.pth'
        torch.save(self.net.state_dict(), save_path)

In [49]:
resnetTrain = ResNet()
#resnetTrain.train(dataloaders_dict)
#resnetTrain.saveWeight()

use in device:  cuda:0
4
tensor([ 2.,  4., 18.,  6.], device='cuda:0')


テスト

In [40]:
class MakeTestDataList():
    
    def __init__(self):
        self.rootpath =  './Test/'
        self.dir = []
        self.datapath = []
        
    def getTest(self):
        for dirpath in glob.glob(self.rootpath+'/*'):
            self.dir.append(dirpath)
            for path in glob.glob(dirpath+'/*.jpg'):
                self.datapath.append(path)
                
        return self.datapath
    

In [41]:
class TestBaseTransform():
    def __init__(self):
        self.size = 256
        self.mean = ((0.5,))
        self.std = ((0.22,))
        self.base_transform = {
            'test':
                transforms.Compose([
                transforms.RandomResizedCrop(self.size, scale=(0.5,1.0)),
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)
                ]),
        }
        
    def __call__(self, img, phase='test'):
        return self.base_transform[phase](img)

In [50]:
test = MakeTestDataList()
path = test.getTest()
transform = TestBaseTransform()
resnetTrain.eval(path, transform)

コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
背景あり
背景あり
背景あり
背景あり
背景あり
背景あり
背景あり
背景あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
背景あり
背景あり
レール
背景あり
背景あり
背景あり
背景あり
背景あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コン

コンクリート
コンクリート
レール
レール
レール
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
背景あり
背景あり
背景あり
レール
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
金枠あり
金枠あり
金枠あり
コンクリート
レール
レール
金枠あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
金枠あり
背景あり
コンクリート
レール
レール
コンクリート
コンクリート
背景あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
金枠あり
金枠あり
背景あり
背景あり
金枠あり
背景あり
背景あり
コンクリート
レール
レール
レール
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
背景あり
背景あり
コンクリート
コンクリート
背景あり
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
コンクリート
コンクリート
コンクリート
レール
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
コンクリー

コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
背景あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
金枠あり
金枠あり
コンクリート
金枠あり
背景あり
コンクリート
コンクリート
コンクリート
背景あり
コンクリート
レール
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
金枠あり
金枠あり
コンクリート
コンクリート
背景あり
レール
背景あり
背景あり
コンクリート
レール
背景あり
レール
レール
背景あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
背景あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
レール
背景あり
コンクリート
コンクリート
レール
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
背景あり
背景あり
背景あり
背景あり
背景あり
背景あり
コンクリート
レール
レール
レール
レール
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
背景あり
コン

コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
背景あり
背景あり
背景あり
コンクリート
レール
レール
コンクリート
コンクリート
レール
金枠あり
レール
レール
レール
コンクリート
背景あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
レール
レール
レール
レール
背景あり
背景あり
レール
コンクリート
コンクリート
金枠あり
コンクリート
レール
コンクリート
コンクリート
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
背景あり
レール
コンクリート
レール
レール
背景あり
コンクリート
コンクリート
金枠あり
コンクリート
コンクリート
金枠あり
金枠あり
コンクリート
レール
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
コンクリート
背景あり
背景あり
背景